In [1]:
#Imports
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
#Visualization Imports
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 
%config InlineBackend.figure_formats = ['retina']
sns.set_style("white")
sns.set_palette('husl')
sns.set_context('talk')

#### Load pickled dataframe

In [3]:
df = pd.read_pickle('./clean_wine_data_w_target.pkl')

In [4]:
df.head()

,description,province,variety,target,target_code
0,aroma include tropical fruit broom brimstone d...,Sicily & Sardinia,White Blend,White Blend from Sicily & Sardinia,305
1,delicate aroma recall white flower citrus pala...,Sicily & Sardinia,White Blend,White Blend from Sicily & Sardinia,305
2,pretty aroma yellow flower stone fruit lead no...,Sicily & Sardinia,White Blend,White Blend from Sicily & Sardinia,305
3,part extend calanìca series grillo viognier bl...,Sicily & Sardinia,White Blend,White Blend from Sicily & Sardinia,305
4,offer heady aroma honeysuckle white stone frui...,Sicily & Sardinia,White Blend,White Blend from Sicily & Sardinia,305


In [5]:
# Create list of all documents
corpus = list(df.description)
corpus = ' '.join(corpus)

In [6]:
# Tokenize all words in joined_corpus
tokens = [word for word in word_tokenize(corpus)]

## Count Vectorizer I

Initial run to create Document-Term Matrix

In [7]:
# Create a document-term matrix using CountVectorizer
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.description)

data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm.head()


,aa,aacacia,aand,aaron,ab,abacela,abacelas,abadal,abandon,abate,...,élevé,élévage,émilion,émilions,étoile,über,überaromatic,überbest,ürzig,ürziger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Takeaways:**
 - Way too many features, need to reduce dimensions
 - Increase list of stop words to target:
   - Domain stop words
   - Words only used in one review
   - Check size of matrix, then potentially remove other most-common words

### Deal with Stop Words

#### Domain Stop Words

In [33]:
domain_stopwords = ['tannins', 'tannin', 'flavor', 'flavors', 'drink', 
                    'wine', 'finish', 'hint', 'fruit', 
                    'note', 'offer', 'aroma', 'style', 
                    'character', 'hint', 'nose', 'bit', 'drinkable', 
                    'palate', 'import', 'bottle', 'varieties', 
                    'year', 'years', 'scented', 'scents', 'wine', 
                    'finish', 'acidity', 'show', 'make', 'age', 'well', 
                    'give', 'touch', 'good', 'vineyard', 'feel', 'like', 
                    'come', 'fine', 'style', 'taste', 'also', 'still', 
                    'one', 'alongside']


#Domain Stopwords
#TANNINS FLAVORS
#FLAVOR DRINK
#WINE FINISH
#HINTS FRUIT
#NOTES OFFERS
#AROMAS STYLE
#CHARACTER HINT
#BIT DRINKABLE
#PALATE IMPORTED

# Domain stopwords adapted from a research paper which used the same basic dataset:
# Martinez, R., et al.; Grapevine: A Wine Prediction Algorithm Using Multi-dimensional Clustering Methods

#### Least-Used Words

In [35]:
# Frequency distribution of words in corpus
fdist = FreqDist(tokens)

In [36]:
# List of words used only once in corpus
used_once = fdist.hapaxes()
len(used_once)

8348

In [37]:
used_once[0:5]

['edèlmio', 'cefalù', 'baccante', 'setback', 'alastro']

In [38]:
# Add new stop words
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS.union(domain_stopwords, used_once)

In [39]:
# This stop words list should cut down on original 30,526-word vocabulary!
len(stop_words)

8685

#### Most Used Words

In [40]:
# Find most commonly-used RELEVANT words (those not in domain_stopwords)

fdist.most_common(100)

[('wine', 67170),
 ('flavor', 58056),
 ('fruit', 53925),
 ('aroma', 32911),
 ('finish', 31397),
 ('palate', 30902),
 ('acidity', 29657),
 ('drink', 29443),
 ('cherry', 28976),
 ('tannin', 28968),
 ('black', 25162),
 ('ripe', 23822),
 ('dry', 23102),
 ('note', 19936),
 ('spice', 19698),
 ('red', 18083),
 ('show', 16523),
 ('berry', 15946),
 ('oak', 15677),
 ('rich', 15298),
 ('fresh', 13949),
 ('offer', 13635),
 ('full', 13608),
 ('blackberry', 13304),
 ('nose', 13270),
 ('plum', 13155),
 ('balance', 12666),
 ('age', 12640),
 ('body', 12339),
 ('apple', 12209),
 ('blend', 11916),
 ('make', 11636),
 ('structure', 11523),
 ('well', 11518),
 ('soft', 11380),
 ('sweet', 11197),
 ('texture', 10877),
 ('dark', 10742),
 ('crisp', 10647),
 ('light', 10534),
 ('give', 10502),
 ('white', 10131),
 ('cabernet', 9518),
 ('raspberry', 9479),
 ('herb', 9389),
 ('vanilla', 9330),
 ('hint', 9109),
 ('year', 9102),
 ('bright', 9019),
 ('citrus', 8939),
 ('pepper', 8672),
 ('touch', 8562),
 ('firm', 8393)

## Count Vectorizer II

In [41]:
# Create a document-term matrix using CountVectorizer
cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(df.description)

data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm.head()

#Less features, still high (14,569)

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['amaro', 'argues', 'argument', 'assertive', 'bad', 'black', 'blend', 'block', 'buster', 'canaiolo', 'ciel', 'comb', 'comes', 'day', 'defined', 'dominates', 'edre', 'elements', 'especially', 'essential', 'ful', 'gentle', 'grained', 'graphite', 'grenache', 'honey', 'kiona', 'molasses', 'monte', 'mourv', 'oak', 'papery', 'power', 'prevail', 'prunes', 'pulciano', 'pushy', 'rhubarb', 'ripe', 'sand', 'sangiovese', 'sign', 'spice', 'spot', 'straightforward', 'structured', 'surprising', 'syrup', 'tea', 'university', 'worthiness', 'yang', 'ying', 'zinfandel'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,aacacia,aaron,abacela,abadal,abandon,abate,abbey,abbott,abbreviate,abeille,...,zweigelt,zédé,àmaurice,élevage,élevé,élévage,émilion,émilions,über,ürziger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Find the words used less than 2 times in descriptions
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=True).head(100)
    top_dict[c]= list(zip(top.index, top.values))

top_dict


KeyboardInterrupt: 

In [ ]:
# Print the top 15 words said by each comedian
for comedian, top_words in top_dict.items():
    print(comedian)
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')

In [ ]:
# Look at the least common words used (those used < 2 times) 
# --> add them to the stop word list

from collections import Counter

# Let's first pull out the top 30 words for each comedian
words = []
for comedian in data.columns:
    top = [word for (word, count) in top_dict[comedian]]
    for t in top:
        words.append(t)
        
words

In [ ]:
# Let's aggregate this list and identify the most common words along with how many routines they occur in
Counter(words).most_common()

In [ ]:
# If more than half of the comedians have it as a top word, exclude it from the list
add_stop_words = [word for word, count in Counter(words).most_common() if count > 6]
add_stop_words

#### Create list of tokenized descriptions

In [5]:
corpus = list(df['description'])

In [6]:
corpus[0:2]

[['aromas',
  'include',
  'tropical',
  'fruit',
  'broom',
  'brimstone',
  'and',
  'dried',
  'herb',
  'the',
  'palate',
  'isnt',
  'overly',
  'expressive',
  'offering',
  'unripened',
  'apple',
  'citrus',
  'and',
  'dried',
  'sage',
  'alongside',
  'brisk',
  'acidity'],
 ['delicate',
  'aromas',
  'recall',
  'white',
  'flower',
  'and',
  'citrus',
  'the',
  'palate',
  'offers',
  'passion',
  'fruit',
  'lime',
  'and',
  'white',
  'peach',
  'with',
  'a',
  'hint',
  'of',
  'mineral',
  'alongside',
  'bright',
  'acidity']]

In [ ]:
# Load stop words
stop_words = stopwords.words('english')

In [ ]:
# Remove stop words from corpus
clean_corpus = []

for desc in corpus:
    desc = [word for word in desc if word not in stop_words]
    clean_corpus.append(desc)

In [ ]:
clean_corpus[0:8]

In [ ]:
# Dataset-specific stop words
wine_stopwords = ['alongside', 'aroma', 'palate', 'offer', 'hint', 'include', 
                  'offering', 'recall', 'pretty', 'nose', 'note', 'lightly', 
                  'part', 'extended', 'series', 'show', 'backed', 'touch', 
                  'flavor', 'provides', 'companion', 'behind', 'mouthfeel', 
                  'could', 'plus', 'open', 'background', 'tone', 'stand', 
                  'isnt', 'expressive', 'mouth', 'wine', 'broad', 'generous', 
                  'term', 'would', 'make', 'tiny', 'blend']

In [ ]:
# Remove stop words from corpus
cleaner_corpus = []

for desc in clean_corpus:
    desc = [word for word in desc if word not in wine_stopwords]
    cleaner_corpus.append(desc)

In [ ]:
cleaner_corpus[0:8]

In [ ]:
# Rejoin lists of words in each description for use in CV & TF-IDF
cleaner_corpus_joined = []

for doc in cleaner_corpus:
    joined = ' '.join(doc)
    cleaner_corpus_joined.append(joined)

In [ ]:
### Then, build model that can determine varietal based on description (use variety as target)

In [ ]:
### ModelSomm: Finally, build model that can determine varietal + Province based off description as a proxy for taste

In [ ]:
## Create Document-Term Matrix from Wine Descriptions

## Count Vectorizer

**Goal: come up with most important vocabulary list for wine descriptions (aka distill wine descriptions down to most important parts) --> Figures**

In [ ]:
cv = CountVectorizer()#ngram_range=(1,2))
X_cv = cv.fit_transform(cleaner_corpus_joined)

print(f"Dimensions of Document-term matrix: {X_cv.toarray().shape}")

In [ ]:
# Checked out the vocab list
# cv.vocabulary_

In [ ]:
#### TF-IDF

In [ ]:
tfidfvec = TfidfVectorizer()#stop_words = 'english')#ngram_range=(1,2))
X_tfidf = tfidfvec.fit_transform(clean_corpus_joined)

print(f"Dimensions of Document-term matrix: {X_tfidf.toarray().shape}")

In [ ]:
### PCA for Scree Plot

In [ ]:
# Choosing number of components with a scree plot
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=200)
pca.fit(X_tfidf)
pcafeatures_train = pca.transform(X_tfidf)

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('# components')
plt.ylabel('explained variance');
plt.title('Scree plot for digits dataset');

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('# components')
plt.ylabel('cumulative explained variance');
plt.title('Cumulative explained variance by PCA for digits');